In [ ]:
%pip install lightning
%pip install optuna
%load_ext tensorboard

In [1]:
dir = 'CNN1D_no' #CHANGE DIR

import os
os.chdir('/home/battisti/Documentos/Acnodnem-Ailiram/Acnodnem-Ailiram')

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

/home/battisti/Documentos/Acnodnem-Ailiram/Acnodnem-Ailiram/CNN1D_no
/home/battisti/Documentos/Acnodnem-Ailiram/Acnodnem-Ailiram


In [6]:
!git clone https://github.com/lucas-battisti/custom-pytorch-lightning-module.git
!mv "custom-pytorch-lightning-module/modules.py" "modules.py"
!rm -rf "custom-pytorch-lightning-module"



!git clone https://github.com/lucas-battisti/pytorch-blocks.git
!mv "pytorch-blocks/blocks.py" "blocks.py"
!rm -rf "pytorch-blocks"

Cloning into 'custom-pytorch-lightning-module'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 25 (delta 11), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (25/25), 20.01 KiB | 931.00 KiB/s, done.
Cloning into 'pytorch-blocks'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 7), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (17/17), 18.58 KiB | 951.00 KiB/s, done.


In [2]:
import lightning as L

from modules import RegressionModule as Reg
from blocks import LinearBlock, Conv1dBlock
from data import Custom_DataModule, CNN1D_Dataset

import torch
from torch import nn
from torch.nn import MaxPool1d, AvgPool1d
from torch.optim.lr_scheduler import ReduceLROnPlateau

import optuna
from optuna.integration import PyTorchLightningPruningCallback

import pandas as pd
from math import floor

import plotly

In [3]:
x = pd.read_csv("data/Dataframe_f.csv", index_col=0).astype('float32')
e = pd.read_csv("data/Dataframe_e_f.csv", index_col=0).astype('float32')
z = pd.read_csv("data/Dataframe_z.csv", index_col=0).astype('float32')

In [9]:
C_IN = 1
L_IN = 16
NUM_EPOCH = 5

dm0 = Custom_DataModule(xez = (x, e, z),
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=False,
                       batch_size=200, num_workers=1)
    
convblock1 = Conv1dBlock(in_channels=C_IN, out_channels=32,
                         kernel_size=3,
                         activation_function=nn.ReLU,
                         pooling_layer=MaxPool1d,
                         pooling_layer_args={"kernel_size": 2})
    
L_convblock1 = convblock1.output_shape(L_IN)['L_out']
    
convblock2 = Conv1dBlock(in_channels=32, out_channels=64,
                         kernel_size=2,
                         activation_function=nn.ReLU,
                         pooling_layer=MaxPool1d,
                         pooling_layer_args={"kernel_size": 2})

L_convblock2 = convblock2.output_shape(L_convblock1)['L_out']
    
dense_input = L_convblock2*64
dense_intermed = int(dense_input*0.75)

l1 = LinearBlock(in_features=dense_input, out_features=dense_intermed,
                 activation_function=nn.ReLU)
    
l2 = LinearBlock(in_features=dense_intermed, out_features=dense_intermed,
                 activation_function=nn.ReLU)
    
l3 = LinearBlock(in_features=dense_intermed, out_features=1)
    
sequential = nn.Sequential(convblock1,
                           convblock2,
                           nn.Flatten(start_dim=1, end_dim=2),
                           l1,
                           l2,
                           l3)

    
m0 = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = dir + '/0')


trainer = L.Trainer(max_epochs=NUM_EPOCH,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m0, datamodule=dm0)
#trainer.test(m0, datamodule=dm0)

trainer.callback_metrics

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 53.1 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
53.1 K    Trainable params
0         Non-trainable params
53.1 K    Total params
0.212     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=5` reached.


{'train_loss': tensor(0.4067), 'val_loss': tensor(0.3727)}

In [8]:
trainer.callback_metrics

{}

In [23]:
def objective(trial: optuna.trial.Trial) -> float:
    NUM_EPOCH = 3
    L_IN = 16
    C_IN = 1
    
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512, 1024, 2048])
    
    dm = Custom_DataModule(xez = (x, e, z),
                           dataset_class=CNN1D_Dataset,
                           version='no', norm=True, #dataset_class kwargs
                           batch_size=batch_size, num_workers=1)
    
    n_conv_layers = trial.suggest_int("n_conv_layers", 1, 3)
    
    sequential = nn.Sequential()
    
    in_channels = [C_IN]
    out_channels = []
    kernel_size = []
    padding = []
    
    batch_norm_conv = trial.suggest_categorical("batch_norm_conv", [False, True])
    if batch_norm_conv:
        batch_norm_conv = nn.BatchNorm1d
    else:
        batch_norm_conv = None
        
    pool_type = trial.suggest_categorical("pool_type", ["max", "avg"])
    if pool_type == "max":
        pool_type = MaxPool1d
    else:
        pool_type = AvgPool1d

    pool_kernel_size = []
    pool_padding = []
        
    length = [L_IN]
    
    
    
    for i in range(n_conv_layers):
        
        out_channels.append(trial.suggest_categorical("n_channels{}".format(i+1), [16, 32, 64, 128]))
        kernel_size.append(trial.suggest_int("kernel_size{}".format(i+1), 1, length[i]))
        padding.append(trial.suggest_int("padding{}".format(i+1), 0, int(floor(kernel_size[i]/2))))

        middle_length = length[i]+2*padding[i]-(kernel_size[i]-1)
        
        pool_kernel_size.append(trial.suggest_int("pool_kernel_size{}".format(i+1), 1, middle_length))
        pool_padding.append(trial.suggest_int("pool_padding{}".format(i+1), 0, int(floor(pool_kernel_size[i]/2))))
        
        sequential = sequential.append(
            Conv1dBlock(in_channels=in_channels[i], out_channels=out_channels[i],
                        kernel_size=kernel_size[i], padding=padding[i],
                        norm_layer=batch_norm_conv, norm_layer_args={"num_features": out_channels[i]},
                        activation_function=nn.ReLU,
                        pooling_layer=pool_type, pooling_layer_args=
                        {"kernel_size":pool_kernel_size[i], "padding":pool_padding[i]})
        )
        

        length.append(int(sequential[i].output_shape(length[i])['L_out']))
        in_channels.append(out_channels[i])
    
    sequential = sequential.append(nn.Flatten(1, 2))
    
    first_input = out_channels[-1]*length[-1]
        
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 3)
    
    dense_layers_sizes = [first_input]

    batch_norm_dense = trial.suggest_categorical("batch_norm_dense", [False, True])
    if batch_norm_dense:
        batch_norm_dense = nn.BatchNorm1d
    else:
        batch_norm_dense = None
    
    for i in range(n_dense_layers):
        
        dense_layers_sizes.append(
            trial.suggest_int("dense_layer_size{}".format(i+1), 1, dense_layers_sizes[i]))
        
        sequential = sequential.append(
            LinearBlock(in_features=dense_layers_sizes[i], out_features=dense_layers_sizes[i+1],
                        norm_layer=batch_norm_dense, norm_layer_args={"num_features": dense_layers_sizes[i+1]},
                        activation_function=nn.ReLU))
        
    sequential = sequential.append(nn.Linear(dense_layers_sizes[-1], 1))
    
    lr = trial.suggest_float("lr", 1e-6, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0, 1e-5)
    
    plateau_factor = trial.suggest_float("plateau_factor", 0, 1)
    
    m = Reg(pytorch_module=sequential,
            loss_func=nn.MSELoss,
            optimizer=torch.optim.Adam, optimizer_args={"lr": lr, "weight_decay": weight_decay},
            lr_scheduler=ReduceLROnPlateau,
            lr_scheduler_args={"mode": 'min', "factor": plateau_factor, "patience": 5, "threshold": 1e-3},
            tb_dir= dir + '/optuna')
    
    trainer = L.Trainer(max_epochs=NUM_EPOCH,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1, callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")], logger = True)
    trainer.fit(m, datamodule=dm)

    return_float = trainer.callback_metrics["val_loss"].item()

    trainer.test(m, datamodule=dm)

    return return_float

In [24]:
pruner = optuna.pruners.HyperbandPruner()

import logging
import sys

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = dir  # Unique identifier of the study.
storage_name = "sqlite:///CNN1D_no/{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name,
                            direction="minimize", pruner=pruner)

study.optimize(objective, n_trials=15, timeout=1800)

[I 2024-03-18 17:27:15,964] A new study created in RDB with name: CNN1D_no


A new study created in RDB with name: CNN1D_no
A new study created in RDB with name: CNN1D_no
A new study created in RDB with name: CNN1D_no
A new study created in RDB with name: CNN1D_no
A new study created in RDB with name: CNN1D_no


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 34    
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
34        Trainable params
0         Non-trainable params
34        Total params
0.000     Total estimated model params size (MB)
/home/battisti/miniconda3/envs/Acnodnem-Ailiram/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/battisti/miniconda3/envs/Acnodnem-Ailiram/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not 

Trial 0 finished with value: 0.48464396595954895 and parameters: {'batch_size': 32, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 64, 'kernel_size1': 5, 'padding1': 1, 'pool_kernel_size1': 6, 'pool_padding1': 0, 'n_dense_layers': 2, 'batch_norm_dense': True, 'dense_layer_size1': 63, 'dense_layer_size2': 33, 'lr': 0.06027496541015467, 'weight_decay': 8.90435556837968e-06, 'plateau_factor': 0.7402629855892903}. Best is trial 0 with value: 0.48464396595954895.
Trial 0 finished with value: 0.48464396595954895 and parameters: {'batch_size': 32, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 64, 'kernel_size1': 5, 'padding1': 1, 'pool_kernel_size1': 6, 'pool_padding1': 0, 'n_dense_layers': 2, 'batch_norm_dense': True, 'dense_layer_size1': 63, 'dense_layer_size2': 33, 'lr': 0.06027496541015467, 'weight_decay': 8.90435556837968e-06, 'plateau_factor': 0.7402629855892903}. Best is trial 0 with value: 0.48464396595954895.
Trial 0 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 33    
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
33        Trainable params
0         Non-trainable params
33        Total params
0.000     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=3` reached.
[I 2024-03-18 17:27:44,936] Trial 1 finished with value: 1.0619359016418457 and parameters: {'batch_size': 64, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 16, 'kernel_size1': 14, 'padding1': 6, 'pool_kernel_size1': 3, 'pool_padding1': 0, 'n_dense_layers': 2, 'batch_norm_dense': False, 'dense_layer_size1': 45, 'dense_layer_size2': 32, 'lr': 0.00072939150594133, 'weight_decay': 2.1898880106674957e-06, 'plateau_factor': 0

...

Trial 12 finished with value: 0.6100358366966248 and parameters: {'batch_size': 256, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'avg', 'n_channels1': 64, 'kernel_size1': 5, 'padding1': 0, 'pool_kernel_size1': 7, 'pool_padding1': 1, 'n_dense_layers': 2, 'batch_norm_dense': True, 'dense_layer_size1': 73, 'dense_layer_size2': 29, 'lr': 0.012006902361893706, 'weight_decay': 6.915602037353307e-06, 'plateau_factor': 0.6308578835598188}. Best is trial 3 with value: 0.48389559984207153.
Trial 12 finished with value: 0.6100358366966248 and parameters: {'batch_size': 256, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'avg', 'n_channels1': 64, 'kernel_size1': 5, 'padding1': 0, 'pool_kernel_size1': 7, 'pool_padding1': 1, 'n_dense_layers': 2, 'batch_norm_dense': True, 'dense_layer_size1': 73, 'dense_layer_size2': 29, 'lr': 0.012006902361893706, 'weight_decay': 6.915602037353307e-06, 'plateau_factor': 0.6308578835598188}. Best is trial 3 with value: 0.48389559984207153.
Tr

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 2     
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)
[I 2024-03-18 17:29:50,119] Trial 13 pruned. Trial was pruned at epoch 1.


Trial 13 pruned. Trial was pruned at epoch 1.
Trial 13 pruned. Trial was pruned at epoch 1.
Trial 13 pruned. Trial was pruned at epoch 1.
Trial 13 pruned. Trial was pruned at epoch 1.
Trial 13 pruned. Trial was pruned at epoch 1.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 7     
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
7         Trainable params
0         Non-trainable params
7         Total params
0.000     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=3` reached.
[I 2024-03-18 17:30:05,337] Trial 14 finished with value: 0.5053030848503113 and parameters: {'batch_size': 32, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 16, 'kernel_size1': 8, 'padding1': 1, 'pool_kernel_size1': 5, 'pool_padding1': 2, 'n_dense_layers': 2, 'batch_norm_dense': False, 'dense_layer_size1': 11, 'dense_layer_size2': 6, 'lr': 0.07452570847977723, 'weight_decay': 7.738394439451061e-06, 'plateau_factor': 0.0

Trial 14 finished with value: 0.5053030848503113 and parameters: {'batch_size': 32, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 16, 'kernel_size1': 8, 'padding1': 1, 'pool_kernel_size1': 5, 'pool_padding1': 2, 'n_dense_layers': 2, 'batch_norm_dense': False, 'dense_layer_size1': 11, 'dense_layer_size2': 6, 'lr': 0.07452570847977723, 'weight_decay': 7.738394439451061e-06, 'plateau_factor': 0.026291166723347525}. Best is trial 3 with value: 0.48389559984207153.
Trial 14 finished with value: 0.5053030848503113 and parameters: {'batch_size': 32, 'n_conv_layers': 1, 'batch_norm_conv': True, 'pool_type': 'max', 'n_channels1': 16, 'kernel_size1': 8, 'padding1': 1, 'pool_kernel_size1': 5, 'pool_padding1': 2, 'n_dense_layers': 2, 'batch_norm_dense': False, 'dense_layer_size1': 11, 'dense_layer_size2': 6, 'lr': 0.07452570847977723, 'weight_decay': 7.738394439451061e-06, 'plateau_factor': 0.026291166723347525}. Best is trial 3 with value: 0.48389559984207153.
Tr